[View in Colaboratory](https://colab.research.google.com/github/heumchri/cityscapesScripts/blob/master/downloadCityscapes.ipynb)

# general

In [0]:
#!kill -9 -1

## utilization monitoring

In [0]:
#requirements for gpu and ram usage

# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize


In [0]:
#gpu and ram usage

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [0]:
#running processes

#!ps -aux
!ps -aux | grep python

In [0]:
#storage usage

!df -h 

# download cityscapes dataset

In [0]:
cd /content/

In [0]:
!mkdir datasets

In [0]:
cd datasets/

In [0]:
!mkdir cityscapes

In [0]:
cd cityscapes/

In [0]:
!wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=heumann.christopher&password=xxx&submit=Login' https://www.cityscapes-dataset.com/login/

In [0]:
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=1
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=3

In [0]:
%%capture
!unzip gtFine_trainvaltest.zip

In [0]:
!rm README license.txt

In [0]:
%%capture
!unzip leftImg8bit_trainvaltest.zip

In [0]:
!rm README cookies.txt index.html index.html.1 license.txt gtFine_trainvaltest.zip leftImg8bit_trainvaltest.zip


# install cityscapesscripts

In [0]:
!pip2 install cython

In [0]:
cd /content/

In [0]:
!git clone https://github.com/heumchri/cityscapesScripts.git

In [0]:
cd cityscapesScripts/

In [0]:
#use 20 classes instead of 2
#!mv /content/cityscapesScripts/cityscapesscripts/helpers/labels_20.py /content/cityscapesScripts/cityscapesscripts/helpers/labels.py

In [0]:
!python2 setup.py build_ext --inplace

In [0]:
import os
os.environ['CITYSCAPES_DATASET'] = '/content/datasets/cityscapes/'

# generate binary training images

In [0]:
!python2 /content/cityscapesScripts/cityscapesscripts/preparation/createTrainIdLabelImgs.py

## visualize results

In [0]:
from IPython.display import Image
#rgb image
Image('/content/datasets/cityscapes/leftImg8bit/train/aachen/aachen_000000_000019_leftImg8bit.png')

In [0]:
#colored ground truth
Image('/content/datasets/cityscapes/gtFine/train/aachen/aachen_000000_000019_gtFine_color.png')

In [0]:
#binary ground truth
Image('/content/datasets/cityscapes/gtFine/train/aachen/aachen_000000_000019_gtFine_labelTrainIds.png')

#calculate class histograms

**old**: class probability -> weights = 1/class probability

1024x2048: 0.22709573, 0.77290427 -> 4.40342925, 1.2938213

512x1024:   0.22700773, 0.77299227 -> 4.40513628, 1.293674

64x128:       0.22824111, 0.77175889 -> 4.38133159, 1.29574148

In [0]:
!pip install torchvision
!pip install Pillow==4.0.0
!pip install PIL
!pip install image

In [0]:
import numpy as np
from PIL import Image
import os
from glob import glob
from torchvision.transforms import Resize

files = []
start_dir = "/content/datasets/cityscapes/gtFine/train"
pattern   = "*labelTrainIds.png"

for dir,_,_ in os.walk(start_dir):
    files += map(Image.open,glob(os.path.join(dir,pattern))) 


##encoder

In [0]:
histo_enc = np.zeros((256,))
j = 0
for image in files:
  j+=1
  if j%100 == 0: 
    print(j)
  image_enc = Resize(64, Image.NEAREST)(image)
  histo_enc = np.add(histo_enc,np.array(image_enc.histogram()))
histo_enc

In [0]:
histo_norm_enc = histo_enc/sum(histo_enc)
histo_norm_enc

In [0]:
normvalue_enc = 1.2

In [0]:
weights_enc = 1/np.log(normvalue_enc+histo_norm_enc)
weights_enc

64x128:       2.35864154, 1.59490491

##decoder

In [0]:
histo_dec = np.zeros((256,))
j = 0
for image in files:
  j+=1
  if j%100 == 0: 
    print(j)
  image_dec = Resize(512, Image.NEAREST)(image)
  histo_dec = np.add(histo_dec,np.array(image_dec.histogram()))
histo_dec

In [0]:
histo_norm_dec = histo_dec/sum(histo_dec)
histo_norm_dec

In [0]:
normvalue_dec = 1.1

In [0]:
weights_dec = 1/np.log(normvalue_dec+histo_norm_dec)
weights_dec

512x1024:  2.81672577,  1.74484367